## GMM 扩容 + 监督学习


In [ ]:
# 确保src目录在Python路径中
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append(os.path.abspath("../"))

# 导入模块
from src.data_utils import filter_anomalous_attributes, identify_attributes, parse_petrel_file
from src.feature_selection import (
    analyze_attribute_correlations,
    analyze_rf_importance_by_group,
)
from src.gmm_clustering import (
    augment_samples_by_pca_mixing,
    encode_cluster_features,
    evaluate_gmm_clusters,
    perform_gmm_clustering,
)
from src.pca_analysis import perform_pca_analysis
from src.regression import (
    build_svr_model,
    predict_with_model,
)

output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# 设置中文字体
plt.rcParams["font.family"] = "SimHei"  # 黑体 SimHei 支持中文
plt.rcParams["axes.unicode_minus"] = False  # 正常显示负号

## 导入地震数据


In [ ]:
data_H6_2_attr = parse_petrel_file("../data/H6-2_attr")

## 导入井震数据

使用 xlsx / csv 数据，注意表名为 Sheet1，注意数据需包含表头

请检查 excel 表头和下面代码中的 selected_columns 是否一致


In [ ]:
file_H6_2_well = "../data/well_processed.xlsx"
data_H6_2_well = pd.read_excel(file_H6_2_well, sheet_name="Sheet1")

# 只选择 Surface 为 H6-2 的行，并丢弃砂厚为 NaN 的行
data_H6_2_well_selected = (
    data_H6_2_well[data_H6_2_well["Surface"] == "H6-2"]
    .replace(-999, np.nan)
    .dropna(subset=["Thickness of facies(1: Fine sand)"])
    .reset_index(drop=True)
)

data_H6_2_well_selected.head()

## 提取共同属性

In [ ]:
# 获取地震属性列表
seismic_attr, _ = identify_attributes("../data/H6-2_attr")

# 提取Excel的属性列表（从第8列开始的所有列）
well_seismic_attr = data_H6_2_well.columns[7:].tolist()

# 计算两个列表的交集
common_attributes = list(set(seismic_attr) & set(well_seismic_attr))

# 打印结果
print(f"地震属性数量: {len(seismic_attr)}")
print(f"Excel属性数量: {len(well_seismic_attr)}")
print(f"共同属性数量: {len(common_attributes)}")
print("\n共同属性列表:")
for attr in common_attributes:
    print(f"- {attr}")

# 筛选出质量良好的属性
good_attributes, anomalous_attributes, attribute_stats = filter_anomalous_attributes(
    seismic_data=data_H6_2_attr,
    well_data=data_H6_2_well_selected,
    common_attributes=common_attributes,
    ratio_threshold=5.0,  # 均值比值阈值
    range_ratio_threshold=10.0,  # 数值范围比值阈值
    std_ratio_threshold=10.0,  # 标准差比值阈值
    output_dir=output_dir,  # 输出图表目录
    verbose=True,  # 打印详细信息
)

print("\n筛选后保留的质量良好属性:")
for attr in good_attributes:
    print(f"- {attr}")

## PCA 降维 + GMM 聚类


In [ ]:
# 对地震属性数据进行PCA降维
pca_results = perform_pca_analysis(
    data=data_H6_2_attr,
    attribute_columns=good_attributes,
    variance_threshold=0.9,
    output_dir=output_dir,
)

In [ ]:
# 首先评估最佳聚类数
gmm_evaluation = evaluate_gmm_clusters(features_pca=pca_results["features_pca"], max_clusters=10, output_dir=output_dir)

# 使用不同的聚类数执行GMM聚类
# 根据BIC/AIC结果选择的最佳聚类数
best_n = gmm_evaluation["best_n_components"]

In [ ]:
best_n = 4  # 如果需要，可以手动设置最佳聚类数

gmm_best = perform_gmm_clustering(
    features_pca=pca_results["features_pca"],
    coords=pca_results["coords_clean"],
    n_clusters=best_n,
    output_dir=output_dir,
)
gmm_best["result_df"].to_csv(os.path.join(output_dir, "gmm_best_clusters.csv"), index=False)

## GMM 中心混合扩充样本（于 PCA 空间）


In [ ]:
# 对井点数据进行PCA降维和GMM聚类
# 注意：这里使用pca_results中的PCA模型和标准化器，但我们需要重新应用到井点数据上

# 确定属性列 - 必须与PCA中使用的列完全一致
attribute_columns = pca_results["features_clean"].columns.tolist()

# 不要只提取特征列，而是保留所有需要的列
X_well = data_H6_2_well_selected.copy()

# 然后对特征列进行预处理
for col in attribute_columns:
    if pd.isna(X_well[col].mean()):
        print(f"列 '{col}' 的均值为NaN，使用0填充")
        X_well[col] = X_well[col].fillna(0)
    else:
        # 使用列均值填充
        X_well[col] = X_well[col].fillna(X_well[col].mean())

# 确保特征列没有剩余的NaN值
if X_well[attribute_columns].isna().any().any():
    print("警告：特征列中仍然存在NaN值，将它们替换为0")
    for col in attribute_columns:
        X_well[col] = X_well[col].fillna(0)

# 对井点数据进行标准化 - 只标准化特征列
X_well_scaled = pca_results["scaler"].transform(X_well[attribute_columns])

# 将井点数据投影到PCA空间
X_well_pca = pca_results["pca"].transform(X_well_scaled)

# 使用已有的GMM模型对井点数据进行聚类
cluster_labels_well = gmm_best["gmm"].predict(X_well_pca)
cluster_probs_well = gmm_best["gmm"].predict_proba(X_well_pca)

# 创建井点数据的聚类结果字典
well_cluster_results = {
    "cluster_labels": cluster_labels_well,
    "cluster_probs": cluster_probs_well,
}

# 使用PCA空间中的中心混合法生成伪样本
augmented_data = augment_samples_by_pca_mixing(
    well_data=X_well,
    pca_model=pca_results["pca"],
    scaler=pca_results["scaler"],
    cluster_results=well_cluster_results,
    attribute_columns=attribute_columns,
    target_column="Thickness of facies(1: Fine sand)",
    min_samples_per_cluster=4,  # 考虑井数据更少，设置更小的阈值
    augmentation_factor=2.0,  # 目标是原始样本数的2倍
    min_target_per_cluster=4,  # 每个聚类至少4个样本
    random_state=42,
    verbose=True,
)

# 保存扩增后的数据
augmented_data.to_csv(os.path.join(output_dir, "augmented_well_data_pca.csv"), index=False)

# 可视化目标值分布
plt.figure(figsize=(12, 6))
sns.histplot(
    data=augmented_data,
    x="Thickness of facies(1: Fine sand)",
    hue="Is_Synthetic",
    element="step",
    bins=15,
    kde=True,
)
plt.title("原始样本和PCA空间中合成样本的目标值分布")
plt.xlabel("砂厚")
plt.savefig(
    os.path.join(output_dir, "pca_augmented_samples_target_dist.png"),
    dpi=300,
    bbox_inches="tight",
)
plt.show()

## 数据整理


### 1. 未扩充的原始井点数据 + One-Hot 编码 + 聚类概率


In [ ]:
# 复制原始井点数据
original_well_data = X_well.copy()

# 添加聚类标签
original_well_data["Cluster"] = cluster_labels_well

# 添加聚类概率
original_well_data["cluster_probs"] = list(cluster_probs_well)

# 应用One-Hot编码
original_well_data_encoded = encode_cluster_features(
    data=original_well_data,
    cluster_column="Cluster",
    drop_original=False,
    prefix="Cluster_",
)

# 保存处理后的数据
output_file = os.path.join(output_dir, "original_well_data_with_clusters.csv")
original_well_data_encoded.to_csv(output_file, index=False)

print(f"原始井点数据（带聚类特征）已保存至: {output_file}")
print(f"数据形状: {original_well_data_encoded.shape}")
print(f"包含的特征列数量: {len(original_well_data_encoded.columns)}")

### 2. 扩充后的数据 + One-Hot 编码


In [ ]:
augmented_well_data_encoded = encode_cluster_features(
    data=augmented_data,
    cluster_column="Cluster",
    drop_original=False,
    prefix="Cluster_",
)

print("\n原始和扩充后井点数据对比:")
print(f"  - 原始井点数据: {len(original_well_data_encoded)} 行")
print(f"  - 扩充后数据: {len(augmented_well_data_encoded)} 行")

### 3. 全体地震数据 + One-Hot 编码 + 聚类概率


In [ ]:
# 直接使用gmm_best["result_df"]，它已经包含了坐标和聚类标签
seismic_with_clusters = gmm_best["result_df"].copy()

# 添加聚类概率
seismic_with_clusters["cluster_probs"] = list(gmm_best["cluster_probs"])

# 将result_df与原始地震数据合并，以保留所有地震属性
# 首先确保坐标列的名称一致
coords_columns = ["X", "Y", "Z"]

# 将地震属性合并到聚类结果中
# 通过坐标列进行合并，这应该是一对一的匹配
seismic_data_full = pd.merge(seismic_with_clusters, data_H6_2_attr, on=coords_columns, how="left")

# 应用One-Hot编码
seismic_data_encoded = encode_cluster_features(
    data=seismic_data_full,
    cluster_column="Cluster",
    drop_original=False,
    prefix="Cluster_",
)

# 保存处理后的数据
output_file = os.path.join(output_dir, "seismic_data_with_clusters.csv")
seismic_data_encoded.to_csv(output_file, index=False)

print(f"全体地震数据（带聚类特征）已保存至: {output_file}")
print(f"数据形状: {seismic_data_encoded.shape}")
print(f"包含的特征列数量: {len(seismic_data_encoded.columns)}")

In [ ]:
print("\n============ 数据整理完成 ============")

# 展示所有准备好的数据集的前几行
print("\n原始井点数据（带聚类特征）前5行:")
display(original_well_data_encoded.head())

print("\n扩充后井点数据（带聚类特征）前5行:")
display(augmented_well_data_encoded.head())

print("\n全体地震数据（带聚类特征）前5行:")
display(seismic_data_encoded.head())

## 属性间相关性分析 + RF importance（SVR）


In [ ]:
# 分析原始特征的相关性
correlated_attribute_groups = analyze_attribute_correlations(
    features_df=pca_results["features_clean"],  # 使用PCA结果中的清理后特征
    method="pearson",  # 使用Pearson相关系数
    corr_threshold=0.85,  # 相关性阈值设为0.85
    output_dir=output_dir,  # 输出目录
    figsize=(16, 14),  # 图像尺寸
)

# 输出每个组的特征数量统计
group_sizes = [len(group) for group in correlated_attribute_groups]
print(f"\n相关属性组统计:")
print(f"  - 总组数: {len(correlated_attribute_groups)}")
print(f"  - 最大组大小: {max(group_sizes)} 属性")
print(f"  - 包含多个属性的组数: {sum(1 for size in group_sizes if size > 1)}")
print(f"  - 平均组大小: {sum(group_sizes) / len(group_sizes):.2f} 属性")

In [ ]:
# 在井点数据上应用随机森林重要性分析
# 使用原始井点数据，不使用扩增数据
selected_features = analyze_rf_importance_by_group(
    well_data=data_H6_2_well_selected,  # 原始井点数据
    attribute_groups=correlated_attribute_groups,  # 前面生成的属性组
    target_column="Thickness of facies(1: Fine sand)",
    top_n=1,  # 每组选择1个最重要的特征
    test_size=0.3,
    n_estimators=100,
    random_state=42,
    output_dir=output_dir,
    verbose=True,
)

## SVR


In [ ]:
# 创建模型比较输出目录
model_comparison_dir = "svr_models_comparison"
if not os.path.exists(model_comparison_dir):
    os.makedirs(model_comparison_dir)

# 1. 原始数据 + selected_features + cluster_probs
svr_original_probs = build_svr_model(
    data=original_well_data_encoded,
    selected_features=selected_features,
    target_column="Thickness of facies(1: Fine sand)",
    use_cluster_probs=False,
    use_onehot=False,
    output_dir=model_comparison_dir,
    filename_prefix="original_with_probs",
)

# 2. 原始数据 + selected_features + one-hot
svr_original_onehot = build_svr_model(
    data=original_well_data_encoded,
    selected_features=selected_features,
    target_column="Thickness of facies(1: Fine sand)",
    use_cluster_probs=False,
    use_onehot=True,
    output_dir=model_comparison_dir,
    filename_prefix="original_with_onehot",
)

# 3. 扩容数据 + selected_features
svr_augmented_base = build_svr_model(
    data=augmented_well_data_encoded,
    selected_features=selected_features,
    target_column="Thickness of facies(1: Fine sand)",
    use_cluster_probs=False,
    use_onehot=False,
    output_dir=model_comparison_dir,
    filename_prefix="augmented_base",
)

# 4. 扩容数据 + selected_features + one-hot
svr_augmented_onehot = build_svr_model(
    data=augmented_well_data_encoded,
    selected_features=selected_features,
    target_column="Thickness of facies(1: Fine sand)",
    use_cluster_probs=False,
    use_onehot=True,
    output_dir=model_comparison_dir,
    filename_prefix="augmented_with_onehot",
)

# 创建模型比较结果表
models = [
    {"name": "原始数据", "results": svr_original_probs},
    {"name": "原始数据+模式识别", "results": svr_original_onehot},
    {"name": "扩容数据", "results": svr_augmented_base},
    {"name": "扩容数据+模式识别", "results": svr_augmented_onehot},
]

# 创建模型比较表
comparison_data = []
for model in models:
    comparison_data.append(
        {
            "模型类型": model["name"],
            "样本数": model["results"]["config"]["data_size"],
            "特征数": model["results"]["config"]["total_features"],
            "R²": model["results"]["metrics"]["r2"],
            "RMSE": model["results"]["metrics"]["rmse"],
            "真实为0预测正确率": model["results"]["metrics"]["true_negative_rate"],
            "真实非0预测正确率": model["results"]["metrics"]["true_positive_rate"],
        }
    )

comparison_df = pd.DataFrame(comparison_data)
comparison_df.to_csv(os.path.join(model_comparison_dir, "svr_models_comparison.csv"), index=False)
print("\nSVR模型性能比较:")
display(comparison_df)

In [ ]:
# 使用每个模型对全体地震数据进行预测
print("\n======== 使用所有模型进行全区域预测 ========")

for model_info in models:
    model_name = model_info["name"]
    model_results = model_info["results"]

    print(f"\n==== 使用{model_name}模型进行预测 ====")
    print(f"模型R²: {model_results['metrics']['r2']:.4f}")

    # 为每个模型创建单独的预测文件名前缀
    filename_prefix = f"model_{model_name.replace('+', '_').replace(' ', '_').lower()}"

    # 进行预测
    predictions = predict_with_model(
        model_results=model_results,
        data=seismic_data_encoded,
        coords_columns=["X", "Y", "Z"],
        output_dir=model_comparison_dir,
        filename_prefix=filename_prefix,
        threshold_zero=0.1,
        verbose=True,
    )